Copyright (c) 2020 Martin Holle. Alle Rechte vorbehalten. Lizensiert unter der MIT-Lizenz.

# Covid-19 Statistics Aachen: Datenaufbereitung

Die rohen Fallzahlen werden zunächst aus dem im ersten Schritt erzeugten Excel-Sheet eingelesen und dann aufbereitet. Anschließend wird eine Reihe von Kennwerten berechnet. Die Ergebnisse werden in eine zweite Excel-Datei exportiert; die zwischengespeicherten Daten werden im folgenden Schritt zur Erstellung von Diagrammen verwendet.

## Benötigte Imports und Initialisierungen

In [1]:
import pandas as pd

import configparser

# Konfiguration einlesen
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

## Einlesen der Excel-Datei und Anzeige der eingelesenen Daten

- Datei: Siehe `config.ini`
- Seite: Siehe `config.ini`
- Einzulesende Spalten: 
  - **A**: Datum im Format 'DD.MM.'
  - **B**: Akkumulierte Anzahl der Infektionen für gesamte Städteregion (inkl. Aachen) als Integerzahl
  - **D**: Akkumulierte Anzahl der Infektionen für die Stadt Aachen als Integerzahl
  - **F**: Akkumulierte Anzahl der Todesfälle durch Covid-19 für gesamte Städteregion (inkl. Aachen) als Integerzahl 
  - **G**: Akkumulierte Anzahl der Genesenen für gesamte Städteregion (inkl. Aachen) als Integerzahl
- Spaltentypen: Spalte A als Datum interpretieren
- Die erste Zeile enthält die Spalten-Header
- Label der Spalten explizit setzen

In [2]:
col_names = ['Summe', 'Summe Aachen', 'Summe Todesfälle', 'Summe genesen', 'Akute Fälle' ]
c19_cases = pd.read_excel(config['Rohdaten']['FileName'], 
                          sheet_name=config['Rohdaten']['SheetName'],
                          usecols='A,C,D,E,F,G', 
                          index_col=0,
                          parse_dates=[0],
                          skiprows=[],
                          names=col_names)

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
Datum,,,,,
2020-03-01,9,4,0,0,9
2020-03-02,10,5,0,0,10
2020-03-03,13,5,0,0,13
2020-03-04,24,8,0,0,24
2020-03-05,33,10,0,0,33
2020-03-06,49,14,0,0,49
2020-03-07,55,15,0,0,55
2020-03-08,57,20,0,0,57
2020-03-09,58,20,0,0,58


## Behandlung fehlender Werte

### Spalte 'Summe Todesfälle'

Am 17.03.2020 wurde das erste Mal in der offiziellen Verlautbarung ein Todesfall erwähnt; für alle Tage vorher wird dieser Spaltenwert zu Null angenommen. Wenn an einem Tag nach dem 17.03. kein Todesfall in der Meldung verzeichnet ist, wird dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Spalte 'Summe genesen'

Das erste Mal wurde in der offiziellen Verlautbarung am 10.03.2020 die Zahl der wieder Genesenen erwähnt, bis zum 31.03. erfolgten nur einzelne Meldungen zu sog. "Freitestungen". Erst danach wird die Zahl der Genesenen regelmäßig in den Meldungen der Krisenstäbe erwähnt. Für alle Tage, für die keine Meldung existiert, wurde dieser Spaltenwert jeweils fortgeschrieben mit dem letzten offiziell gemeldeten; dadurch kann es zu Sprüngen in der Statistik kommen.

### Hinzufügen von Tagen, für die keine Fallzahlen vorliegen

Ab dem 01.05.2020 sind die Krisenstäbe von Städteregion und der Stadt Aachen dazu übergegangen, neue Fallzahlen nur noch an normalen Wochentagen zu veröffentlichen; ab Juni wurden nur noch 3x pro Woche die Fallzahlen bekannt gegeben. Damit enthalten die aus dem Excelsheet eingelesenen Zeitreihen tageweise Lücken, insbesondere für die Wochenenden. Diese werden in diesem Schritt aufgefüllt, um die Berechnung gleitender Mittelwerte und Summen später zu vereinfachen.

In [3]:
# Hinzufügen von Tagen, für die keine Fallzahlen vorliegen: Dataframe neu indizieren und fehlende Werte ergänzen

method = config['Kennzahlen']['Impute.Method']

# Fehlende Spaltenwerte auffüllen, indem der jeweils letzte ('ffill') oder nächste ('bfill') gültige Wert 
# fortgeschrieben wird
if method == 'bfill' or method == 'ffill' or method == 'nearest':
    c19_cases = c19_cases.reindex(pd.date_range(c19_cases.index[0], c19_cases.index[-1]), method=method)

c19_cases

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle
2020-03-01,9,4,0,0,9
2020-03-02,10,5,0,0,10
2020-03-03,13,5,0,0,13
2020-03-04,24,8,0,0,24
2020-03-05,33,10,0,0,33
2020-03-06,49,14,0,0,49
2020-03-07,55,15,0,0,55
2020-03-08,57,20,0,0,57
2020-03-09,58,20,0,0,58
2020-03-10,61,21,0,6,55


## Korrektur der Spaltenformate
Die Fallzahlen wurden wegen fehlender Werte teilweise als Fließkommazahlen (NaN erfordert ein Float zu Darstellung) importiert, es handelt sich aber in allen Spalten um Ganzzahlen.

In [4]:
# Explizite Umwandlung aller Spalten (außer Index) nach Integer
c19_cases = c19_cases.astype('int32')

## Qualitätskontrolle
Die Summe der Fälle (für die Städteregion) abzüglich der Summe der Todesfälle und abzüglich der Summe der Genesenen muss gleich der Anzahl der akuten Fälle sein. Die resultierende Liste ist leer, wenn alle Zahlen korrekt sind.

In [5]:
c19_cases[c19_cases['Summe'] - c19_cases['Summe Todesfälle'] - c19_cases['Summe genesen'] != c19_cases['Akute Fälle']]

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle


## Berechnung zusätzlicher Spalten

- Absolutes und prozentuales Wachstum (oder Rückgang) der aktiven Fälle berechnen
- Anzahl der Neuinfektionen pro Tag (für Stadt Aachen und Städteregion) berechnen
- Anzahl der neuen Todesfälle pro Tag (für Städteregion) berechnen
- Anzahl der neu Genesenen pro Tag (für Städteregion) berechnen
- Gleitendes Mittel (über `Glaettung.Intervall` Tage) für die obigen Kennwerte berechnen
- Summen der Fälle bezogen auf 100.000 Einwohner (für Stadt Aachen und Städteregion) berechnen
- Summe der Neuinfektionen der letzten 7 Tage pro 100.000 Einwohner („7-Tage-Inzidenz“ für Stadt Aachen und Städteregion) berechnen

In [6]:
Einwohner_StädteRegion = int(config['Kennzahlen']['Einwohner.Region'])
Einwohner_StadtAachen = int(config['Kennzahlen']['Einwohner.Aachen'])
Glättungsintervall = int(config['Kennzahlen']['Glaettung.Intervall'])

def change_per_cent(x):
    """Evaluates and returns the change of the active cases per day.

    Für Berechnung der prozentualen Änderung der aktiven Fälle pro Tag per apply():
    Berechnet die Differenz zwischen dem ersten und letzten Wert des als Array übergebenem 'Rolling Window' 
    und dividiert dann durch den ersten Wert.
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0]) / x[0] * 100

def diff(x):
    """Für Berechnung von neuen Infektionen / Genesenen / Todesfällen pro Tag per apply():
    Berechnet die Differenz zwischen dem ersten und letzten Wert des als Array übergebenem 'Rolling Window'
    """
    if x.size == 1:
        return 0
    else:
        return (x[-1] - x[0])
    
# Aktive Fälle
c19_cases['Änderung Akute Fälle'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Änderung Akute Fälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle'].rolling(Glättungsintervall, center=True).mean()
c19_cases['Änderung Akute Fälle [%]'] = c19_cases['Akute Fälle'].rolling(2, min_periods=1).apply(change_per_cent, raw=True)
c19_cases['Änderung Akute Fälle (MW/{0}T) [%]'.format(str(Glättungsintervall))] = c19_cases['Änderung Akute Fälle [%]'].rolling(Glättungsintervall, center=True).mean()

# Neuinfektionen
c19_cases['Neuinfektionen'] = c19_cases['Summe'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen Aachen'] = c19_cases['Summe Aachen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neuinfektionen (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neuinfektionen'].rolling(Glättungsintervall, center=True).mean()

# Neue Todesfälle
c19_cases['Neue Todesfälle'] = c19_cases['Summe Todesfälle'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Todesfälle (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Todesfälle'].rolling(Glättungsintervall, center=True).mean()

# Neue Genesene
c19_cases['Neue Genesene'] = c19_cases['Summe genesen'].rolling(2, min_periods=1).apply(diff, raw=True).astype('int32')
c19_cases['Neue Genesene (MW/{0}T)'.format(str(Glättungsintervall))] = c19_cases['Neue Genesene'].rolling(Glättungsintervall, center=True).mean()

# Fälle pro 100.000 Einwohner in den letzten 7 Tagen
c19_cases['Summe pro 100000'] = c19_cases['Summe'] / Einwohner_StädteRegion * 100000
c19_cases['Summe Aachen pro 100000'] = c19_cases['Summe Aachen'] / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000 Aachen'] = c19_cases['Neuinfektionen Aachen'].rolling(7).sum() / Einwohner_StadtAachen * 100000
c19_cases['Neuinfektionen 7T/100000'] = c19_cases['Neuinfektionen'].rolling(7).sum() / Einwohner_StädteRegion * 100000


## Zwischenergebnis in Excel-Datei speichern

In [7]:
c19_cases.to_excel(config['Kennzahlen']['FileName'], 
                   sheet_name=config['Kennzahlen']['SheetName'],
                   index_label='Datum')

c19_cases.tail(14)

,Summe,Summe Aachen,Summe Todesfälle,Summe genesen,Akute Fälle,Änderung Akute Fälle,Änderung Akute Fälle (MW/7T),Änderung Akute Fälle [%],Änderung Akute Fälle (MW/7T) [%],Neuinfektionen,Neuinfektionen Aachen,Neuinfektionen (MW/7T),Neue Todesfälle,Neue Todesfälle (MW/7T),Neue Genesene,Neue Genesene (MW/7T),Summe pro 100000,Summe Aachen pro 100000,Neuinfektionen 7T/100000 Aachen,Neuinfektionen 7T/100000
2020-09-12,2429,1186,103,2271,55,9,7.714286,19.565217,16.554025,17,8,12.000000,0,0.000000,8,4.285714,437.291278,479.424367,10.105910,8.101321
2020-09-13,2429,1186,103,2271,55,0,9.428571,0.000000,17.145803,0,0,14.714286,0,0.000000,0,5.285714,437.291278,479.424367,8.893201,7.201174
2020-09-14,2429,1186,103,2271,55,0,9.142857,0.000000,14.641773,0,0,15.714286,0,0.000000,0,6.571429,437.291278,479.424367,8.893201,7.201174
2020-09-15,2477,1210,103,2288,86,31,10.714286,56.363636,16.070345,48,24,19.714286,0,0.000000,17,9.000000,445.932687,489.126041,17.786401,15.122465
2020-09-16,2503,1224,103,2297,103,17,9.428571,19.767442,13.275314,26,14,17.285714,0,0.000000,9,7.857143,450.613450,494.785350,21.424529,18.543023
2020-09-17,2522,1232,103,2309,110,7,8.428571,6.796117,12.448867,19,8,19.857143,0,0.142857,12,11.285714,454.034008,498.019242,21.828765,19.803228
2020-09-18,2550,1248,103,2326,121,11,8.428571,10.000000,12.448867,28,16,19.857143,0,0.142857,17,11.285714,459.074829,504.487024,28.296548,24.844050
2020-09-19,2550,1248,103,2326,121,0,3.571429,0.000000,4.020979,0,0,15.285714,0,0.142857,0,11.571429,459.074829,504.487024,25.062657,21.783551
2020-09-20,2568,1252,104,2350,114,-7,0.857143,-5.785124,0.939659,18,4,13.857143,1,0.142857,24,12.857143,462.315357,506.103970,26.679602,25.024079
2020-09-21,2568,1252,104,2350,114,0,1.285714,0.000000,1.279401,0,0,14.142857,0,0.142857,0,12.714286,462.315357,506.103970,26.679602,25.024079
